## Naive method prompt

### Prompts

The prompts for the naive and guided approaches are in table 1 of the paper.

Naive prompt:

```
In the task of <task on task-dataset>, a trained black-box classifier correctly predicted the label ‘<yi>’ for the following text. Generate a counterfactual explanation by making minimal changes to the input text, so that the label changes from ‘<yi>’ to ‘<ycf >’. Use the following definition of ‘counterfactual explanation’: “A counterfactual explanation reveals what should have been different in an instance to observe a diverse outcome." Enclose the generated text within <new> tags.\n—\nText: <xi>.
```

<task on task-dataset> denotes the task description such as “natural language inference on the SNLI dataset". <xi> denotes the input text. <yi> denotes the label output by the black-box classifier for the input text <xi>. <ycf > denotes the desired label for the counterfactual text (any label other than <yi>).


In [23]:
from openai import OpenAI
client = OpenAI()


'<new>An uninspiring love story that brings out the latent 15-year-old romantic in everyone.</new>'

In [43]:
import re

task = "sentiment analysis on the SST-2 dataset"

def generate_fizle_counterfactual(original_text, args):
    original_label = args["original_label"]
    cf_label = 0 if original_label == 1 else 1

    prompt = f"""In the task of {task}, a trained black-box classifier correctly predicted the label '{original_label}' for the following text. Generate a counterfactual explanation by making minimal changes to the input text, so that the label changes from '{original_label}' to '{cf_label}'. Use the following definition of 'counterfactual explanation': "A counterfactual explanation reveals what should have been different in an instance to observe a diverse outcome." Enclose the generated text within <new> tags.
    -
    Text: {original_text}"""

    completion = client.chat.completions.create(
        model="gpt-4-turbo",
        messages=[
            {"role": "system", "content": prompt}
        ]
    )

    output = completion.choices[0].message.content

    counterfactual_text = re.search("<new>(.*?)</new>", output).group(1)
    return counterfactual_text


In [44]:
original_text = "an idealistic love story that brings out the latent 15-year-old romantic in everyone ."
args = {
    "original_label": 1
}

counterfactual_text = generate_fizle_counterfactual(original_text, args)

print(f"{'original_text'.ljust(20)}: {original_text}")
print(f"{'counterfactual_text'.ljust(20)}: {counterfactual_text}")

original_text       : an idealistic love story that brings out the latent 15-year-old romantic in everyone .
counterfactual_text : An unrealistic love story that disappoints the latent 15-year-old romantic in everyone.


In [45]:
x = 2

def print_var():
    print(x)

print_var()

2
